# Experiment 07: Tiny CNN - Underfitting Analysis
## Objective: Demonstrate underfitting with an intentionally small model and analyze capacity limitations

In [ ]:
# Install necessary packages
!pip install wandb -q
!pip install kaggle -q

In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import wandb
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Mount Google Drive (optional - for saving results)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Setup kaggle directory
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download FER2013 dataset from Kaggle
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

# Extract the dataset
!unzip -q challenges-in-representation-learning-facial-expression-recognition-challenge.zip
!ls

In [ ]:
# Initialize W&B
wandb.login()
run = wandb.init(
    project="fer-challenge",
    name="exp07-tiny-cnn-underfitting",
    config={
        "architecture": "Tiny CNN (Underfitting)",
        "dataset": "FER2013",
        "epochs": 50,  # More epochs to see if tiny model can learn
        "batch_size": 64,
        "learning_rate": 0.001,
        "weight_decay": 0.0,  # No regularization
        "dropout": 0.0,  # No dropout
        "batch_norm": False,  # No batch norm
        "conv_blocks": 2,  # Very few blocks
        "conv_channels": [16, 32],  # Very small channels
        "fc_sizes": [64],  # Very small FC layer
        "num_classes": 7,
        "model_purpose": "demonstrate_underfitting"
    }
)

In [ ]:
# Load and explore the data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")
print("\nTraining data columns:", train_df.columns.tolist())
print("\nEmotion distribution:")
print(train_df['emotion'].value_counts().sort_index())

icml_df = pd.read_csv('icml_face_data.csv')

# Split ICML data based on 'Usage'
icml_train = icml_df[icml_df[' Usage'] == 'Training']
icml_test = icml_df[icml_df[' Usage'].isin(['PublicTest', 'Other'])]

# Drop the 'Usage' column (not needed after splitting)
icml_train = icml_train.drop(columns=[' Usage'])
icml_test = icml_test.drop(columns=[' Usage'])

# Merge datasets
train_df = pd.concat([train_df, icml_train], ignore_index=True)
test_df = pd.concat([test_df, icml_test], ignore_index=True)

# **Added data type check and filtering**
print("\nChecking 'pixels' column data types...")
initial_train_rows = len(train_df)
initial_test_rows = len(test_df)

train_df = train_df[train_df['pixels'].apply(lambda x: isinstance(x, str))]
test_df = test_df[test_df['pixels'].apply(lambda x: isinstance(x, str))]

print(f"Removed {initial_train_rows - len(train_df)} rows from training set due to non-string 'pixels'.")
print(f"Removed {initial_test_rows - len(test_df)} rows from test set due to non-string 'pixels'.")

# Shuffle the merged datasets (optional but recommended)
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Output shapes and emotion distribution
print("\nMerged Train shape (after filtering):", train_df.shape)
print("Merged Test shape (after filtering):", test_df.shape)

print("\nEmotion distribution in merged train set:")
print(train_df['emotion'].value_counts().sort_index())

print("\nEmotion distribution in merged test set:")
print(test_df['emotion'].value_counts().sort_index())

In [ ]:
# Visualize sample images
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

fig, axes = plt.subplots(2, 4, figsize=(12, 6))
axes = axes.ravel()

for i in range(8):
    idx = np.random.randint(0, len(train_df))
    pixels = train_df.iloc[idx]['pixels']
    emotion = train_df.iloc[idx]['emotion']

    # Convert pixel string to array and reshape
    pixels = np.array([int(pixel) for pixel in pixels.split(' ')], dtype=np.uint8)
    pixels = pixels.reshape(48, 48)

    axes[i].imshow(pixels, cmap='gray')
    axes[i].set_title(f'{emotion_labels[emotion]}')
    axes[i].axis('off')

plt.suptitle('Sample Images from FER2013 Dataset')
plt.tight_layout()
wandb.log({"sample_images": wandb.Image(plt)})
plt.show()

In [ ]:
# Custom Dataset Class
class FERDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        pixels = self.data.iloc[idx]['pixels']
        emotion = self.data.iloc[idx]['emotion']

        # Convert pixel string to numpy array
        pixels = np.array([int(pixel) for pixel in pixels.split(' ')], dtype=np.float32)
        pixels = pixels / 255.0  # Normalize to [0, 1]

        # For CNN, reshape to (1, 48, 48) - single channel
        pixels = pixels.reshape(1, 48, 48)

        return torch.tensor(pixels), torch.tensor(emotion, dtype=torch.long)

In [ ]:
# Create datasets
full_dataset = FERDataset(train_df)

# Split into train and validation
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)

In [ ]:
# Tiny CNN Model - Intentionally Too Small
class TinyCNN(nn.Module):
    def __init__(self, num_classes=7):
        super(TinyCNN, self).__init__()

        # Only 2 convolutional blocks with very few filters
        # Block 1: Very small number of filters
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, padding=2)  # Large kernel, few filters
        self.pool1 = nn.MaxPool2d(4, 4)  # Aggressive pooling

        # Block 2: Still very small
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, padding=2)
        self.pool2 = nn.MaxPool2d(4, 4)  # More aggressive pooling

        # Very small fully connected layer
        # After two 4x4 poolings: 48/4/4 = 3x3
        self.fc1 = nn.Linear(32 * 3 * 3, 64)  # Very small hidden layer
        self.fc2 = nn.Linear(64, num_classes)

        # Simple activation
        self.relu = nn.ReLU()

        # Calculate total parameters
        self.total_params = sum(p.numel() for p in self.parameters())

    def forward(self, x):
        # Conv Block 1
        x = self.relu(self.conv1(x))
        x = self.pool1(x)

        # Conv Block 2
        x = self.relu(self.conv2(x))
        x = self.pool2(x)

        # Flatten
        x = x.view(x.size(0), -1)

        # FC layers
        x = self.relu(self.fc1(x))
        x = self.fc2(x)

        return x

In [ ]:
# Initialize model, loss, optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = TinyCNN().to(device)
print(f"Total parameters: {model.total_params:,}")
print(f"Model size comparison:")
print(f"  - Tiny CNN: {model.total_params:,} parameters")
print(f"  - Typical CNN: ~500K-2M parameters")
print(f"  - Large CNN: ~10M+ parameters")

criterion = nn.CrossEntropyLoss()
# No weight decay - we want to see pure underfitting
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Log model architecture to W&B
wandb.watch(model, log='all')

In [ ]:
# Print model architecture
print("Tiny Model Architecture:")
print("=" * 50)
print("Convolutional Layers:")
print("  Conv1: 1 -> 16 filters (5x5 kernel)")
print("  Pool1: 4x4 pooling (aggressive)")
print("  Conv2: 16 -> 32 filters (5x5 kernel)")
print("  Pool2: 4x4 pooling (aggressive)")
print("\nFully Connected Layers:")
print("  FC1: 288 -> 64 (very small hidden layer)")
print("  FC2: 64 -> 7")
print("\nTotal Layers: 4 (2 conv + 2 fc)")
print(f"Total Parameters: {model.total_params:,}")
print("=" * 50)

# Compare with typical model sizes
for name, param in model.named_parameters():
    print(f"{name:15} {param.shape} -> {param.numel():,} params")

In [ ]:
# Training function
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    progress_bar = tqdm(loader, desc='Training')
    for inputs, labels in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Update progress bar
        progress_bar.set_postfix({
            'loss': loss.item(),
            'acc': 100 * correct / total
        })

    epoch_loss = running_loss / len(loader)
    epoch_acc = 100 * correct / total

    return epoch_loss, epoch_acc

In [ ]:
# Validation function
def validate_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        progress_bar = tqdm(loader, desc='Validation')
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Update progress bar
            progress_bar.set_postfix({
                'loss': loss.item(),
                'acc': 100 * correct / total
            })

    epoch_loss = running_loss / len(loader)
    epoch_acc = 100 * correct / total

    return epoch_loss, epoch_acc, all_predictions, all_labels

In [ ]:
# Training loop - Extended to see if tiny model can eventually learn
train_losses = []
train_accs = []
val_losses = []
val_accs = []
best_val_acc = 0

# Train for more epochs to see the underfitting behavior
num_epochs = 50

print("Starting training of intentionally tiny model...")
print("Expected behavior: Both training and validation accuracy should plateau at low values")
print("This demonstrates underfitting due to insufficient model capacity.\n")

for epoch in range(num_epochs):
    print(f'\nEpoch {epoch+1}/{num_epochs}')
    print('-' * 50)

    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Validate
    val_loss, val_acc, predictions, labels = validate_epoch(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    # Log to W&B
    wandb.log({
        'epoch': epoch + 1,
        'train_loss': train_loss,
        'train_acc': train_acc,
        'val_loss': val_loss,
        'val_acc': val_acc,
        'learning_rate': optimizer.param_groups[0]['lr'],
        'overfitting_gap': train_acc - val_acc,
        'underfitting_indicator': max(train_acc, val_acc) < 40  # Low accuracy indicates underfitting
    })

    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
    print(f'Gap: {train_acc - val_acc:.2f}% (small gap indicates underfitting)')

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_tiny_cnn_model.pth')
        print(f'New best model saved with validation accuracy: {val_acc:.2f}%')
        
    # Check for underfitting signs
    if epoch > 10:
        recent_train_acc = np.mean(train_accs[-5:])
        recent_val_acc = np.mean(val_accs[-5:])
        if recent_train_acc < 40 and recent_val_acc < 40:
            print("UNDERFITTING DETECTED: Both training and validation accuracy are low")

In [ ]:
# Plot training history emphasizing underfitting patterns
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Loss plot
ax1.plot(train_losses, label='Train Loss', linewidth=2, color='blue')
ax1.plot(val_losses, label='Val Loss', linewidth=2, color='orange')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Validation Loss\n(Notice both remain high - underfitting)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Accuracy plot with reference lines
ax2.plot(train_accs, label='Train Acc', linewidth=2, color='blue')
ax2.plot(val_accs, label='Val Acc', linewidth=2, color='orange')
ax2.axhline(y=100/7, color='red', linestyle='--', alpha=0.7, label='Random Guess (14.3%)')
ax2.axhline(y=50, color='green', linestyle='--', alpha=0.7, label='Reasonable Performance (50%)')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Training and Validation Accuracy\n(Both plateau at low values - underfitting)')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 60)

plt.tight_layout()
wandb.log({"underfitting_analysis": wandb.Image(plt)})
plt.show()

In [ ]:
# Detailed underfitting analysis
print("\n" + "=" * 60)
print("UNDERFITTING ANALYSIS")
print("=" * 60)

# Calculate key metrics
final_train_acc = train_accs[-1]
final_val_acc = val_accs[-1]
max_train_acc = max(train_accs)
max_val_acc = max(val_accs)
overfitting_gaps = [train_accs[i] - val_accs[i] for i in range(len(train_accs))]
avg_gap = np.mean(overfitting_gaps)

print(f"\nPerformance Metrics:")
print(f"  - Final Training Accuracy: {final_train_acc:.2f}%")
print(f"  - Final Validation Accuracy: {final_val_acc:.2f}%")
print(f"  - Maximum Training Accuracy: {max_train_acc:.2f}%")
print(f"  - Maximum Validation Accuracy: {max_val_acc:.2f}%")
print(f"  - Average Train-Val Gap: {avg_gap:.2f}%")

print(f"\nUnderfitting Indicators:")
print(f"  - Low absolute performance: {max_val_acc < 40}")
print(f"  - Small train-val gap: {abs(avg_gap) < 5}")
print(f"  - Training accuracy plateau: {max_train_acc - train_accs[10] < 5 if len(train_accs) > 10 else 'N/A'}")

# Compare with random baseline
random_accuracy = 100 / 7  # 7 classes
print(f"\nBaseline Comparison:")
print(f"  - Random Guess Accuracy: {random_accuracy:.2f}%")
print(f"  - Model vs Random: +{max_val_acc - random_accuracy:.2f} percentage points")
print(f"  - Improvement over random: {(max_val_acc / random_accuracy - 1) * 100:.1f}%")

In [ ]:
# Plot learning curves with underfitting indicators
plt.figure(figsize=(12, 8))

# Create subplot for detailed analysis
plt.subplot(2, 2, 1)
plt.plot(train_accs, 'b-', linewidth=2, label='Training')
plt.plot(val_accs, 'r-', linewidth=2, label='Validation')
plt.axhline(y=random_accuracy, color='gray', linestyle='--', alpha=0.7, label='Random')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Learning Curves - Underfitting Pattern')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 2)
plt.plot(overfitting_gaps, 'g-', linewidth=2)
plt.axhline(y=0, color='black', linestyle='-', alpha=0.5)
plt.xlabel('Epoch')
plt.ylabel('Train - Val Accuracy (%)')
plt.title('Overfitting Gap (Small = Underfitting)')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 3)
plt.plot(train_losses, 'b-', linewidth=2, label='Training')
plt.plot(val_losses, 'r-', linewidth=2, label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curves - Both Remain High')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 4)
# Show model capacity vs problem complexity
model_capacity = model.total_params
problem_complexity = len(train_df)  # Dataset size as proxy
capacity_ratio = model_capacity / problem_complexity * 1000  # Scale for visualization

plt.bar(['Model\nCapacity', 'Problem\nComplexity\n(scaled)'], 
        [model_capacity, problem_complexity/1000], 
        color=['red', 'blue'], alpha=0.7)
plt.ylabel('Count')
plt.title('Model Capacity vs Problem Complexity')
plt.yscale('log')

plt.tight_layout()
wandb.log({"underfitting_detailed_analysis": wandb.Image(plt)})
plt.show()

In [ ]:
# Load best model for final evaluation
model.load_state_dict(torch.load('best_tiny_cnn_model.pth'))
_, _, final_predictions, final_labels = validate_epoch(model, val_loader, criterion, device)

# Confusion matrix
cm = confusion_matrix(final_labels, final_predictions)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=emotion_labels,
            yticklabels=emotion_labels)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Tiny CNN (Underfitting)\nNote: Poor performance across all classes')
wandb.log({"confusion_matrix": wandb.Image(plt)})
plt.show()

# Analyze confusion matrix for underfitting signs
print("\nConfusion Matrix Analysis:")
print(f"  - Diagonal sum (correct predictions): {np.trace(cm)}")
print(f"  - Total predictions: {np.sum(cm)}")
print(f"  - Accuracy from CM: {np.trace(cm)/np.sum(cm)*100:.2f}%")

# Check if model is biased towards certain classes
predicted_class_counts = np.sum(cm, axis=0)
actual_class_counts = np.sum(cm, axis=1)
print(f"\nClass Prediction Analysis:")
for i, emotion in enumerate(emotion_labels):
    print(f"  {emotion}: Predicted {predicted_class_counts[i]}, Actual {actual_class_counts[i]}")

In [ ]:
# Classification report
print("\nClassification Report:")
print("=" * 70)
report = classification_report(final_labels, final_predictions,
                             target_names=emotion_labels,
                             output_dict=True)
print(classification_report(final_labels, final_predictions, target_names=emotion_labels))

# Log per-class metrics to W&B
for emotion in emotion_labels:
    wandb.log({
        f"{emotion}_precision": report[emotion]['precision'],
        f"{emotion}_recall": report[emotion]['recall'],
        f"{emotion}_f1": report[emotion]['f1-score']
    })

# Analyze per-class performance for underfitting
print("\nPer-class Performance Analysis:")
for emotion in emotion_labels:
    f1 = report[emotion]['f1-score']
    if f1 < 0.3:
        print(f"  - {emotion}: Very poor performance (F1={f1:.3f}) - indicates underfitting")
    elif f1 < 0.5:
        print(f"  - {emotion}: Poor performance (F1={f1:.3f}) - likely underfitting")
    else:
        print(f"  - {emotion}: Reasonable performance (F1={f1:.3f})")

In [ ]:
# Model capacity analysis
print("\n" + "=" * 60)
print("MODEL CAPACITY ANALYSIS")
print("=" * 60)

# Calculate theoretical model capacity
input_size = 48 * 48  # Input image size
num_classes = 7
dataset_size = len(train_df)

print(f"\nProblem Characteristics:")
print(f"  - Input Dimensions: {input_size:,} pixels")
print(f"  - Number of Classes: {num_classes}")
print(f"  - Training Samples: {dataset_size:,}")
print(f"  - Complexity Estimate: High (facial expressions)")

print(f"\nModel Characteristics:")
print(f"  - Total Parameters: {model.total_params:,}")
print(f"  - Parameters per Input Pixel: {model.total_params/input_size:.3f}")
print(f"  - Parameters per Training Sample: {model.total_params/dataset_size:.3f}")
print(f"  - Bottleneck Layer Size: 64 neurons")

# Rule of thumb comparisons
print(f"\nCapacity Assessment:")
params_per_sample = model.total_params / dataset_size
if params_per_sample < 0.1:
    print(f"  - SEVERELY UNDERCAPACITATED: {params_per_sample:.3f} params/sample")
elif params_per_sample < 1:
    print(f"  - UNDERCAPACITATED: {params_per_sample:.3f} params/sample")
else:
    print(f"  - Adequate capacity: {params_per_sample:.3f} params/sample")

print(f"\nRecommendations to Fix Underfitting:")
print(f"  1. Increase model size (more filters, layers)")
print(f"  2. Reduce aggressive pooling")
print(f"  3. Add more convolutional layers")
print(f"  4. Increase fully connected layer sizes")
print(f"  5. Remove regularization (if any)")
print(f"  6. Train for more epochs")
print(f"  7. Try different architectures (ResNet, etc.)")

In [ ]:
# Save final model and log to W&B
torch.save(model.state_dict(), 'final_tiny_cnn_model.pth')
wandb.save('final_tiny_cnn_model.pth')
wandb.save('best_tiny_cnn_model.pth')

# Summary statistics emphasizing underfitting
summary_stats = {
    "final_train_accuracy": train_accs[-1],
    "final_val_accuracy": val_accs[-1],
    "best_val_accuracy": best_val_acc,
    "max_train_accuracy": max(train_accs),
    "overfitting_gap": train_accs[-1] - val_accs[-1],
    "avg_overfitting_gap": np.mean(overfitting_gaps),
    "total_parameters": model.total_params,
    "macro_f1_score": report['macro avg']['f1-score'],
    "weighted_f1_score": report['weighted avg']['f1-score'],
    "improvement_over_random": best_val_acc - random_accuracy,
    "underfitting_confirmed": best_val_acc < 40,
    "parameters_per_sample": model.total_params / len(train_df),
    "training_epochs": len(train_losses)
}

wandb.log(summary_stats)

In [ ]:
# Final summary
print("\n" + "=" * 70)
print("EXPERIMENT SUMMARY: TINY CNN - UNDERFITTING DEMONSTRATION")
print("=" * 70)
print(f"\nModel Architecture:")
print(f"  - Intentionally Small Design")
print(f"  - 2 Convolutional Layers (16, 32 filters)")
print(f"  - Aggressive Pooling (4x4)")
print(f"  - Small FC layer (64 neurons)")
print(f"  - Total Parameters: {model.total_params:,}")
print(f"  - No Regularization Applied")

print(f"\nUnderfitting Evidence:")
print(f"  - Final Training Accuracy: {train_accs[-1]:.2f}%")
print(f"  - Final Validation Accuracy: {val_accs[-1]:.2f}%")
print(f"  - Best Validation Accuracy: {best_val_acc:.2f}%")
print(f"  - Small Train-Val Gap: {np.mean(overfitting_gaps):.2f}%")
print(f"  - Low Absolute Performance: {best_val_acc < 40}")
print(f"  - Improvement over Random: +{best_val_acc - random_accuracy:.2f}pp")
print(f"  - Macro F1-Score: {report['macro avg']['f1-score']:.3f}")

print(f"\nKey Insights:")
print(f"  - Model lacks sufficient capacity for the task")
print(f"  - Both training and validation accuracy plateau at low values")
print(f"  - Minimal overfitting due to insufficient model complexity")
print(f"  - Poor performance across all emotion classes")
print(f"  - Training for more epochs doesn't help")

wandb.finish()